In [1]:
import re
from math import log as ln
def loadData(file):
    with open(file) as f:
        res = f.read()
        sents = list ( filter ( lambda x: len(x) > 1, re.split('\n|\.|\!|\?|\...', res)))
        avg_len = sum ( list ( map ( lambda x : len(x.split()), sents )))
    return res.split(), avg_len/len(sents)

In [2]:
def load1gram(data):
    dict_gram = {}
    for word in data:
        if word not in dict_gram:
            dict_gram[word] = 0
        dict_gram[word] += 1
    return dict_gram

In [3]:
def load2gram(data):
    dict_gram = {}
    key = data[0]
    for word in data[1:]:
        if key not in dict_gram:
            dict_gram[key] = {}
        if word not in dict_gram[key]:
            dict_gram[key][word] = 0
        dict_gram[key][word] += 1
        key = word
    return dict_gram

In [4]:
def load3gram(data):
    dict_gram = {}
    key = (data[0],data[1])
    for word in data[2:]:
        #print(key)
        if key not in dict_gram:
            dict_gram[key] = {}
        if word not in dict_gram[key]:
            dict_gram[key][word] = 0
        dict_gram[key][word] += 1
        tmp = (key[1],word)
        key = tmp
    return dict_gram

In [5]:
data_orzeszkowa, avg_o = loadData("korpus_orzeszkowej.txt")
data_prus, avg_p = loadData("korpus_prusa.txt")
data_sienkiewicz, avg_s = loadData("korpus_sienkiewicza.txt")
#print(avg_o)
#print(data_orzeszkowa[:100])
gram1_orzeszkowa = load1gram(data_orzeszkowa)
gram2_orzeszkowa = load2gram(data_orzeszkowa)
gram3_orzeszkowa = load3gram(data_orzeszkowa)

gram1_prus = load1gram(data_prus)
gram2_prus = load2gram(data_prus)
gram3_prus = load3gram(data_prus)

gram1_sienkiewicz = load1gram(data_sienkiewicz)
gram2_sienkiewicz = load2gram(data_sienkiewicz)
gram3_sienkiewicz = load3gram(data_sienkiewicz)

feature = {'P' : [gram1_prus,gram2_prus,gram3_prus,avg_p], 'O' : [gram1_orzeszkowa,gram2_orzeszkowa,gram3_orzeszkowa,avg_o], 'S' : [gram1_sienkiewicz,gram2_sienkiewicz,gram3_sienkiewicz,avg_s]}

In [18]:
def NaiveBayes(test):
    prob = {i : 0 for i in 'POS'}
    data_test, avg_t = loadData(test)
    gram1_test = load1gram(data_test)
    for w in gram1_test:
        for i in 'POS':
            if w in feature[i][0]: # unigram w is in gram1_...
                prob[i] += ln(feature[i][0][w] / len(feature[i][0]))
            else:
                prob[i] += ln(1.0 / len(feature[i][0]))
                
    key = data_test[0]
    for w in data_test[1:]:
        for i in 'POS':
            if key in feature[i][1]:
                if w in feature[i][1][key]:
                    prob[i] += ln(feature[i][1][key][w] / feature[i][0][key] )
                    #prob[i] += ln(feature[i][1][key][w] / len (feature[i][1]) )
            else:
                prob[i] += (0.3)*ln(1.0 / len(feature[i][0])) 
        key = w
        
    key = (data_test[0],data_test[1])
    for w in data_test[2:]:
        for i in 'POS':
            if key in feature[i][2]:
                if w in feature[i][2][key]:
                    prob[i] += ln(feature[i][2][key][w] / feature[i][1][key[0]][key[1]] )
                    #prob[i] += ln(feature[i][2][key][w] / len(feature[i][2]) )
            else:
                prob[i] += (0.7)*ln(1.0 / len(feature[i][0])) 
        tmp = (key[1],w)
        key = tmp
            
    for i in 'POS':
        prob[i] += ln( feature[i][3] / (feature[i][3] + abs (feature[i][3] - avg_t)  ))
        
    return max(prob.items(), key = lambda x: x[1])[0] , prob
        

In [19]:
for i in range(0,42,2):
    f = f"testy1/test_prusa{i}.txt"
    res = NaiveBayes(f)
    if res[0] == 'P':
        print(res[0])
    else:
        print(res)
print('\n\n')
for i in range(1,23,2):
    f = f"testy1/test_orzeszkowej{i}.txt"
    res = NaiveBayes(f)
    if res[0] == 'O':
        print(res[0])
    else:
        print(res)
print('\n\n')
for i in range(1,55,2):
    f = f"testy1/test_sienkiewicza{i}.txt"
    res = NaiveBayes(f)
    if res[0] == 'S':
        print(res[0])
    else:
        print(res)

P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P



O
O
O
('S', {'P': -15931.412418476895, 'O': -15494.951623367708, 'S': -15457.475514343894})
('S', {'P': -15174.195797178023, 'O': -15005.09511988723, 'S': -14842.318234274057})
('S', {'P': -15002.735144027336, 'O': -14740.149391194973, 'S': -14701.601491313382})
('S', {'P': -15209.58017658191, 'O': -14972.875711072322, 'S': -14830.259642407102})
('S', {'P': -15751.484097648448, 'O': -15651.77173028726, 'S': -15420.429169249992})
O
O
O



S
S
S
S
S
S
S
S
('P', {'P': -13726.033339251173, 'O': -14100.032324160653, 'S': -13767.404035697646})
S
('P', {'P': -13102.86641212393, 'O': -13600.324554718007, 'S': -13167.305564294598})
S
S
S
S
S
S
('P', {'P': -12583.795054810662, 'O': -12988.44634647526, 'S': -12676.31561618292})
S
S
S
S
('P', {'P': -12710.79547504893, 'O': -12960.880732758738, 'S': -12740.176223894803})
S
S
S
S
